In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
import matplotlib
import signale
import numpy as np
import matplotlib.pyplot as pl
import spectrum as sp
import pickle as pkl
import scipy.signal as scsig
import pywt
import colormaps as mycmps
import signale.tools as tools
import os
import itertools
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from sklearn.cluster import KMeans
from matplotlib.mlab import PCA as mlabPCA
from mtspec import mtspec

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
def MA(array,ord=2):
    ma = np.array([array[ii-ord/2:ii+ord/2].mean() for ii in range(array.size) if ii >= ord ],dtype=float)
    ma = np.insert(ma,0,array[0:ord/2])
    ma = np.append(ma,array[-(ord/2)-1:-1])
    return ma
def zScore(array):
    return(array - np.average(array))/ np.std(array)
def mySpecgram(sig,minFreq = None,maxFreq = None,overlap=None,
               windowSize= None,NFFT = None,Fs = None,method=None):
    if not overlap:
        overlap = 48
    kIdx = 0
    timeSlices = []
    timeCenters = []
    while kIdx < sig.size:
        #timeEdges = np.arange(0,Nbins*windowSize,windowSize)
        timeSlices.append([kIdx,kIdx+windowSize])
        kIdx += windowSize - overlap
    Pxx = []
    #print timeSlices
    for item in timeSlices:
        timeCenters.append((item[0]+np.diff(item)/2)[0])
        data = sig[item[0]:item[1]]
        if data.size < windowSize:
            ddd = windowSize - data.size
            if ddd%2 ==0:
                data = np.lib.pad(data, (ddd/2,ddd/2), 'constant', constant_values=(0,0))
            else:
                data = np.lib.pad(data, (ddd/2,ddd/2+1), 'constant', constant_values=(0,0))
        if method == 'yule':
            p = sp.pyule(data, 550, norm='biased', NFFT=NFFT,sampling=Fs)
        elif method =='fft':
            p = sp.Periodogram(data,NFFT=NFFT,sampling=Fs)
        p();
        freqz = np.array(p.frequencies())
        Pxx.append(p.psd)
    Pxx = np.array(Pxx).T
    if not minFreq:
        minFreq = freqz.min()
    if not maxFreq:
        maxFreq = freqz.max()
    indexstart = np.where(freqz >= minFreq)[0][0]
    indexend   = np.where(freqz <= maxFreq)[0][-1]
    freqz = freqz[indexstart:indexend]
    Pxx = Pxx[indexstart:indexend]
    timeCenters = np.array(timeCenters)
    return timeCenters,freqz,Pxx
def multiTaperSpecgrm(sig,minFreq = None,maxFreq = None,overlap=None,
               windowSize= None,NFFT = None,Fs = None,method=None):
    if not overlap:
        overlap = 48
    kIdx = 0
    timeSlices = []
    timeCenters = []
    while kIdx < sig.size:
        #timeEdges = np.arange(0,Nbins*windowSize,windowSize)
        timeSlices.append([kIdx,kIdx+windowSize])
        kIdx += windowSize - overlap
    Pxx = []
    for item in timeSlices: 
        timeCenters.append((item[0]+np.diff(item)/2)[0])
        data = sig[item[0]:item[1]]
        if data.size < windowSize:
            ddd = windowSize - data.size
            if ddd%2 ==0:
                data = np.lib.pad(data, (ddd/2,ddd/2), 'constant', constant_values=(0,0))
            else:
                data = np.lib.pad(data, (ddd/2,ddd/2+1), 'constant', constant_values=(0,0))
        spec, freqz = mtspec(data, 1.0, 2.0, number_of_tapers=5,quadratic=False,nfft=4096)
        freqz*=2e3
        Pxx.append(spec)
    Pxx = np.array(Pxx).T
    if not minFreq:
        minFreq = freqz.min()
    if not maxFreq:
        maxFreq = freqz.max()
    indexstart = np.where(freqz >= minFreq)[0][0]
    indexend   = np.where(freqz <= maxFreq)[0][-1]
    freqz = freqz[indexstart:indexend]
    Pxx = Pxx[indexstart:indexend]
    timeCenters = np.array(timeCenters)
    return timeCenters,freqz,Pxx

In [20]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-12-16_Day4/'
ddayx = animalPath.split('/')[-2].split('_')[-1]
#print dday
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat3903Lesion/dataFiles/sleepDB/'

In [34]:
lfpPaths = []
for lfp in tools.locate('CSC1.lfp',animalPath):#animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = np.array(sorted(lfpPaths))[[0,2]]
ImmoDBpaths = []
for db in tools.locate('I*.db',dataFolder):
    if db[1].find(ddayx)>0:
        ImmoDBpaths.append(os.path.join(db[0],db[1]))
lfpPaths = sorted(lfpPaths)
ImmoDBpaths = sorted(ImmoDBpaths)
lfpPaths,ImmoDBpaths

(['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-12-16_Day4/01-sleep1/CSC1.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-12-16_Day4/03-sleep2/CSC1.lfp'],
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat3903Lesion/dataFiles/sleepDB/ImmPeriods-Rat3903-Day4-01-pre.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat3903Lesion/dataFiles/sleepDB/ImmPeriods-Rat3903-Day4-03-post.db'])

In [35]:
SleepEpochsDB = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
for item,jtem in zip(ImmoDBpaths[:],lfpPaths[:]):
    print 'LOADING %s' %jtem
    ##Loading...
    expDate = animalPath.split('/')[-2].split('_')[-1]
    session = jtem.split('/')[-2].split('-')[-1]
    session = session[:-1]+'0'+session[-1]
    animal = [prt for prt in jtem.split('/') if not prt.find('Rat')][0]
    SleepTimeIndex = pd.read_pickle(item)
    lfp = pkl.load(open(jtem,'rb'))
    t,frq,Pxx =  multiTaperSpecgrm(lfp.signal,minFreq=0.1,maxFreq=200,windowSize=2048,Fs=2000,overlap=10,NFFT=4096)
    timeInSec = t/2e3+lfp.timeAxis[0]/1e3
    #Pxx = np.clip(Pxx,0,Pxx.mean()+1.4*Pxx.std())
    pxxFlat = np.array([Pxx[:,ii]*frq for ii in range(Pxx.shape[1])]).T
    for jjj in range(pxxFlat.shape[0]):
        pxxFlat[jjj,:] = MA(pxxFlat[jjj,:],12)
    
    ##Grouping into different freqency bands, delta,theta,gamma
    Dmean = frq[frq<5].mean()
    Tmean = frq[np.logical_and(frq>6,frq<11)].mean()
    Gmean = frq[np.logical_and(frq>11,frq<50)].mean()
    Rmean = frq[frq>160].mean()
    Dbounds = np.where(frq<5)[0][[0,-1]]
    Tbounds = np.where(np.logical_and(frq>6,frq<11))[0][[0,-1]]
    Gbounds = np.where(np.logical_and(frq>11,frq<50))[0][[0,-1]]
    Rbounds = np.where(frq>160)[0][[0,-1]]
    DavgPwr = Pxx[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TavgPwr = Pxx[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GavgPwr = Pxx[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    RavgPwr = Pxx[Rbounds[0]:Rbounds[-1]+1,:].mean(axis=0)
    ##################################
    DpwrNormal = pxxFlat[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TpwrNormal = pxxFlat[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GpwrNormal = pxxFlat[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    RpwrNormal = pxxFlat[Rbounds[0]:Rbounds[-1]+1,:].mean(axis=0)
    ####Creating a filter for asleep times
    asleepMask = np.zeros(DavgPwr.size)
    for idx,row in SleepTimeIndex[SleepTimeIndex['type']=='asleep'].iterrows():
        asleepMask += np.logical_and(timeInSec>row.t_start,timeInSec<row.t_stop)
    ###Clustering states using k-means
    pwrData = np.array([DpwrNormal*asleepMask,TpwrNormal*asleepMask,GpwrNormal*asleepMask,RpwrNormal*asleepMask]).T
    #pwrPCA= mlabPCA(pwrData)
    if 1e2*(1-asleepMask.sum()/asleepMask.size) >10:
        print 'Splitting in 3 clusters!!!'
        Klusters   = KMeans(n_clusters=3).fit_predict(pwrData)
        #KlustersPC = KMeans(n_clusters=3).fit_predict(pwrPCA.Y)
        Klusters +=8
        awkKluster = Klusters[asleepMask==0][0]
        KlusterTags = np.array([8,9,10])
        aspKlusters = KlusterTags[KlusterTags!=awkKluster]
        Klusters[np.where(Klusters==awkKluster)[0]]-=(awkKluster-2)
        for iclust,clust in enumerate(aspKlusters):
            Klusters[np.where(Klusters==clust)[0]]-=(clust-iclust)
    else:
        print 'Splitting in 2 clusters!!!'
        Klusters   = KMeans(n_clusters=2).fit_predict(pwrData)
    
    print set(Klusters)
    #######################################################################################################################pl.plot(k2)
    UpIdx   = np.where(Klusters==1)[0]
    DownIdx = np.where(Klusters==0)[0]
    SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
    SingletIdxD = np.where(np.diff(UpIdx)  ==2)[0]
    while SingletIdxU.size or SingletIdxD.size:
        print SingletIdxU.size,SingletIdxD.size
        if UpIdx.size > DownIdx.size:
            SingletIdxU = np.int32(DownIdx[SingletIdxU]  + np.ones(SingletIdxU.size))
            np.put(Klusters,SingletIdxU,np.zeros(SingletIdxU.size))
            SecondFlip =False
        else:
            SingletIdxD = np.where(np.diff(UpIdx)==2)[0]
            SingletIdxD  = np.int32(UpIdx[SingletIdxD]  + np.ones(SingletIdxD.size))
            np.put(Klusters,SingletIdxD,np.ones(SingletIdxD.size))
            SecondFlip =True
        if SecondFlip:
            SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
            SingletIdxU  = np.int32(DownIdx[SingletIdxU]  + np.ones(SingletIdxU.size))
            np.put(Klusters,SingletIdxU,np.zeros(SingletIdxU.size))
        else:
            SingletIdxD = np.where(np.diff(UpIdx)==2)[0]
            SingletIdxD  = np.int32(UpIdx[SingletIdxD]  + np.ones(SingletIdxD.size))
            np.put(Klusters,SingletIdxD,np.ones(SingletIdxD.size))
        del SecondFlip
        print set(Klusters)
        ###########222#######
        ###
        ###########222#######
        K1ratio = (DpwrNormal[Klusters==1] / TpwrNormal[Klusters==1]).mean()
        K0ratio = (DpwrNormal[Klusters==0] / TpwrNormal[Klusters==0]).mean()
        K11ratio = (DavgPwr[Klusters==1] / TavgPwr[Klusters==1]).mean()
        K00ratio = (DavgPwr[Klusters==0] / TavgPwr[Klusters==0]).mean()
        print K1ratio,K0ratio
        print K11ratio,K00ratio
        if K1ratio<K0ratio:
            Ones = np.where(Klusters==1)[0]
            Zers = np.where(Klusters==0)[0] 
            print 'Cluster labels fliped!!!'
            Klusters[Ones] -= 1
            Klusters[Zers] += 1
        k1 = Klusters.copy()
        ###
        # from now on SWS episodes are Kluster ==1
        ###
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==3)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==3)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.zeros(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.zeros(dupletIdx2nd.size))
        print set(Klusters)
        ##########333############
        ###Now chunks of length 3!
        ##########333############
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==4)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==4)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        try:
            dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
            dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        except IndexError:
            print 'It\'s a bit marginal!'
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        print set(Klusters)
        ##########444############
        ###Now chunks of length 4!
        ##########444############
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==5)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        dupletIdx4th = np.int32(swsIdx[dupletIdx] +4* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        np.put(Klusters,dupletIdx4th,np.ones(dupletIdx4th.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==5)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        try:
            dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
            dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
            dupletIdx4th = np.int32(swsIdx[dupletIdx] +4* np.ones(dupletIdx.size))
        except IndexError:
            print 'It\'s a bit marginal!'
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        np.put(Klusters,dupletIdx4th,np.ones(dupletIdx4th.size))
                
        UpIdx   = np.where(Klusters==1)[0]
        DownIdx = np.where(Klusters==0)[0]
        SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
        SingletIdxD = np.where(np.diff(UpIdx)  ==2)[0]
    print 100.0*np.where(Klusters==1)[0].size/Klusters.size
    ####################################################
    SleepEpochs = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
    print set(Klusters)
    borders = np.union1d([0],np.where(pd.rolling_std(Klusters,2)>0)[0])
    Kids = ['REM','SWS','AWAKE']
    for iib,brd in enumerate(borders):
        if brd < borders[-1]:
            chunk = Klusters[brd:borders[iib+1]]
            df = pd.DataFrame([[timeInSec[brd:borders[iib+1]][0],timeInSec[brd:borders[iib+1]][-1],Kids[chunk[0]],session,expDate,animal]],
                              columns=['t0','t1','epoch','session','date','animal'])
            SleepEpochs = SleepEpochs.append(df,ignore_index=True)
        else:
            chunk = Klusters[brd:]
            df = pd.DataFrame([[timeInSec[brd:][0],timeInSec[brd:][-1],Kids[chunk[0]],session,expDate,animal]],
                             columns=['t0','t1','epoch','session','date','animal'])
            SleepEpochs = SleepEpochs.append(df,ignore_index=True)
    SleepEpochsDB = SleepEpochsDB.append(SleepEpochs,ignore_index=True)
    #############################################################################################
    f,ax=pl.subplots(1,1)
    ax.pcolormesh(timeInSec,frq,np.log10(pxxFlat),cmap='magma')
    ax.plot((lfp.timeAxis)/1e3,10*zScore(lfp.signal)+70,lw=1)
    for idx,row in SleepEpochs.iterrows():
        if row.epoch == 'SWS':
            ax.axvspan(row.t0,row.t1,ymin=0.9,ymax=0.95,color='r',alpha=0.5)
        elif row.epoch == 'REM':
            ax.axvspan(row.t0,row.t1,ymin=0.8,ymax=0.85,alpha=0.5)
    ax.plot(timeInSec,10*Klusters,lw=3)
    #############################################################################################

LOADING /home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-12-16_Day4/01-sleep1/CSC1.lfp
Splitting in 2 clusters!!!
set([0, 1])
5 4
set([0, 1])
0.337192 0.280191
1.05711 1.03444
set([0, 1])
set([0, 1])
83.0800114712
set([0, 1])
LOADING /home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-12-16_Day4/03-sleep2/CSC1.lfp
Splitting in 3 clusters!!!
set([0, 1, 2])
5 4
set([0, 1, 2])
0.25984 0.292149
0.98877 0.935541
Cluster labels fliped!!!
set([0, 1, 2])
set([0, 1, 2])
54.2198100407
set([0, 1, 2])


In [36]:
SleepEpochsDB_toSave = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
SleepEpochsDB_toSave = SleepEpochsDB_toSave.append(SleepEpochsDB[SleepEpochsDB.t1-SleepEpochsDB.t0 > 10],ignore_index=True)
SleepEpochsDB_toSave.to_pickle(dataFolder+'SleepEpoch-'+animal+'-'+expDate+'.db')
SleepEpochsDB_toSave

,t0,t1,epoch,session,date,animal
0,0.512,235.901,SWS,sleep01,Day4,Rat3903
1,236.920,255.262,REM,sleep01,Day4,Rat3903
2,260.357,296.022,REM,sleep01,Day4,Rat3903
3,297.041,394.865,SWS,sleep01,Day4,Rat3903
4,395.884,422.378,REM,sleep01,Day4,Rat3903
5,426.454,456.005,REM,sleep01,Day4,Rat3903
6,457.024,612.931,SWS,sleep01,Day4,Rat3903
7,613.950,640.444,REM,sleep01,Day4,Rat3903
8,641.463,655.729,SWS,sleep01,Day4,Rat3903
9,656.748,671.014,REM,sleep01,Day4,Rat3903


In [25]:
pd.read_pickle(ImmoDBpaths[1])

,t_start,t_stop,type
0,5556.073195,5733.284575,asleep
1,5733.284575,5888.339485,awake
2,5888.339485,6379.397632,asleep
3,6379.397632,6395.413888,awake
4,6395.413888,6901.754048,asleep
5,6901.754048,6906.093246,awake
6,6906.093246,7206.660068,asleep
7,7206.660068,7219.406215,awake
8,7219.406215,7341.894320,asleep
9,7341.894320,7360.778049,awake


In [22]:
pl.plot(asleepMask)
pl.ylim(-0.1,1.1)

(-0.1, 1.1)

In [29]:
asleepMask.sum()/asleepMask.size

0.45563093622795114

In [30]:
pl.plot(DpwrNormal*asleepMask)

In [31]:
pl.plot(asleepMask)

In [33]:
lfp.timeAxis/1e3

array([  6750.262398,   6750.262898,   6750.263398, ...,  10505.268898,
        10505.269398,  10505.269898])